# Paso 1: Importar las librerías y cargar el los datos del archivo

In [1]:
# Importando todas las librerías que se van a utilizar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Cargar los datos
df = pd.read_csv("users_behavior.csv")
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


En esta sección se importaron todas las librerías necesarias: las de pandas, numpy, los tres diferentes modelos, la de accuracy y la que permite la segmentación de los datos (train_test_split). No hay valores ausentes, por lo que pasé a la segmentación.

# Paso 2: Segmentación de los datos en un conjunto de entrenamiento, uno de validación y uno de prueba

In [5]:
# Separar características y objetivos
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']

In [6]:
# Segmentación en 60% de entrenamiento y 40% el resto
features_train, features_temp, target_train, target_temp = train_test_split( features, target, test_size=0.4, random_state=12345)

In [7]:
# Segmentación 20-20 restante
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, target_temp,test_size=0.5, random_state=12345)

In [8]:
print(target_train.count())
print(target_valid.count())
print(target_test.count())

1928
643
643


En este caso, el modelo de prueba no existe, lo que signfica que tenemos que tener tres segmentos de datos: uno para el conjunto de entrenamiento, para el de validación y el de prueba. Siguiendo lo aprendido en las lecciones anteriores, usualmente, el tamaño del conjunto de validación y del de prueba son iguales. Esto da como resultado una proporción de datos fuente de 3:1:1. Es por eso que se dividieron los datos en 60%-20%-20%.

# Paso 3: Evaluando la calidad de diferentes modelos cambiando los hiperparámetros

**Arbol de decisión**

In [9]:
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid,predictions_valid)
    print("max_depth=",depth,": ", end=" ")
    print(accuracy)

max_depth= 1 :  0.7542768273716952
max_depth= 2 :  0.7822706065318819
max_depth= 3 :  0.7853810264385692
max_depth= 4 :  0.7791601866251944
max_depth= 5 :  0.7791601866251944
max_depth= 6 :  0.7838258164852255
max_depth= 7 :  0.7822706065318819
max_depth= 8 :  0.7791601866251944
max_depth= 9 :  0.7822706065318819
max_depth= 10 :  0.7744945567651633


**Bosque aleatorio**

In [10]:
for n in range(10,101,10):
    model = RandomForestClassifier(random_state=12345, n_estimators=n)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid,predictions_valid)
    print("n_estimators=",n,": ", end=" ")
    print(accuracy)

n_estimators= 10 :  0.7853810264385692
n_estimators= 20 :  0.7869362363919129
n_estimators= 30 :  0.7838258164852255
n_estimators= 40 :  0.7838258164852255
n_estimators= 50 :  0.7916018662519441
n_estimators= 60 :  0.7853810264385692
n_estimators= 70 :  0.7791601866251944
n_estimators= 80 :  0.7822706065318819
n_estimators= 90 :  0.7838258164852255
n_estimators= 100 :  0.7853810264385692


**Regresión logística**

In [11]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train,target_train)
predictions_valid = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions_valid)

print(f"Regresión Logística Exactitud={accuracy:.4f}")

Regresión Logística Exactitud=0.7589


Se entrenaron los tres modelos y se hizo la predicción para el cojunto de validación: el de árbol de decisión, bosque aleatorio y regresión logística. En el caso del árbol de decisión, se hizo un ciclo del 1 al 10 para encontrar la mejor opción, siendo 3 la profundidad donde se obtuvo mayor accuracy en el modelo, de 0.7853. Para el del bosque aleatorio, se fue cambiando el hiperparámetro n_estimators, cambiando de 10 en 10 hasta llegar a 100. Se encontró que el n_estimators que signficaba mayor exactitud para el modelo era de 50, con un accuracy de 0.7916. Finalmente, se entrenó con regresión logística y se obtuvo un modelo con una accuracy de 0.7589. A todos se les puso un random_state para que se mantuviera la misma división de los datos. El paso siguiente es la comprobación de la calidad del modelo usando el conjunto de prueba.  

# Paso 4: Comprobando la calidad del modelo usando el conjunto de prueba

**Arbol de decisión**

In [12]:
tree_model = DecisionTreeClassifier(max_depth=3, random_state=12345)
tree_model.fit(features_train, target_train)
predictions_tree = tree_model.predict(features_test)
tree_accuracy = accuracy_score(target_test, predictions_tree)
print(f"Árbol de Decisión en test: {tree_accuracy:.4f}")

Árbol de Decisión en test: 0.7792


**Bosque aleatorio**

In [13]:
forest_model = RandomForestClassifier(n_estimators=50, random_state=12345)
forest_model.fit(features_train, target_train)
predictions_forest = forest_model.predict(features_test)
forest_accuracy = accuracy_score(target_test, predictions_forest)
print(f"Bosque Aleatorio en test: {forest_accuracy:.4f}")

Bosque Aleatorio en test: 0.7932


**Regresión logística**

In [14]:
log_model = LogisticRegression(random_state=12345, solver='liblinear')
log_model.fit(features_train, target_train)
predictions_log = log_model.predict(features_test)
log_accuracy = accuracy_score(target_test, predictions_log)
print(f"Regresión Logística en test: {log_accuracy:.4f}")

Regresión Logística en test: 0.7403


Una vez que los modelos se entrenaron, se probaron para el conjunto de prueba. Se obtuvo que el que mejor accuracy tiene es el bosque aleatorio, con un 0.7932

# Paso 5: Tarea adicional, hacer una prueba de cordura del modelo

In [15]:
# Creación del DataFrame con valores mínimos y máximos
test_data = pd.DataFrame({
    'calls': [df['calls'].min(), df['calls'].max()],
    'minutes': [df['minutes'].min(), df['minutes'].max()],
    'messages': [df['messages'].min(), df['messages'].max()],
    'mb_used': [df['mb_used'].min(), df['mb_used'].max()]
})

# Realizar predicciones con el modelo ya entrenado
test_predictions = forest_model.predict(test_data)

# Mostrar resultados
test_data['Predicción is_ultra'] = test_predictions
print(test_data)

   calls  minutes  messages   mb_used  Predicción is_ultra
0    0.0     0.00       0.0      0.00                    1
1  244.0  1632.06     224.0  49745.73                    1


En el último paso, se realizó una prueba de cordura utilizando valores mínimos y máximos de consumo. El modelo predice que todos los casos corresponden a is_ultra = 1, lo que indica que podría no estar diferenciando correctamente usuarios de bajo consumo. Esto indica que, a pesar de tener un buen accuracy, nuevas técnicas deben ser implementadas para mejorar el equilibrio de clases.

# Conclusiones

La modificación de hiperparámetros para encontrar la mayor accuracy es de suma importancia, pues así se encuntra la opción que mejor se adapta. Al final del proyecto surgió otro tema muy interesante a analizar: el equilibrio de clases.